In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import plotly.express as px

from sampling_handler.misc import py_helpers

# 1 Load augmented dataset

In [ ]:
# change to your project name here
project_name = 'YOUR_PROJECT_NAME'

# locate out folder of dataset augmented files
results_folder = Path(f'/home/sepal-user/module_results/esbae/{project_name}/04_Dataset_Augmentation/Landsat')

# glob all files in the data augmentation output folder
files = results_folder.glob('*geojson')

# prepare for parallel execution
files = [[str(file), False] for file in files]

# read files in parallel nad put the in a list
dfs = py_helpers.run_in_parallel(
    py_helpers.geojson_to_gdf,
    files,
    workers=os.cpu_count()*2,
    parallelization='processes'
)

# concatenate output
df = pd.concat(dfs)

# 2 Display Single Timeseries

In [ ]:
from scipy.optimize import curve_fit

def harmonic_sine_func(t, A, phi, C):
    omega = 2 * np.pi / 365.25  # Angular frequency for yearly seasonality
    return A * np.sin(omega * t + phi) + C

# use FFT to get initial parameters
initial_A = np.sqrt(2) * (np.max(ts) - np.min(ts))
initial_phi = 0
initial_C = np.mean(ts)

p0 = [initial_A, initial_phi, initial_C] 
print(p0)

x_data = pd.to_numeric(dates) / 10 ** 9 / 60 / 60 / 24
fit_params, _ = curve_fit(harmonic_sine_func, x_data, ts, p0=p0)

print(fit_params)
fitted_A, fitted_phi, fitted_C = fit_params

fitted_values = harmonic_sine_func(x_data, fitted_A, fitted_phi, fitted_C)
fitted_values

actual_values = ts 
residuals = actual_values - fitted_values
squared_residuals = residuals**2
mean_squared_error = np.mean(squared_residuals)
rmse = np.sqrt(mean_squared_error)

num_observations = len(actual_values)  # Number of observations in your dataset
normalized_rmse = rmse / np.sqrt(num_observations)
print(rmse, normalized_rmse)

fake_dates = pd.date_range('2000-01-01', '2023-06-01')
fake_x = pd.to_numeric(fake_dates) / 10 ** 9 / 60 / 60 / 24
fake_fitted_values = harmonic_sine_func(fake_x, fitted_A, fitted_phi, fitted_C)

In [ ]:
# define point and band
pid_column = 'point_id'
point_of_interest = 123519
band = 'ndfi'

# get dates and ts
dates = pd.to_datetime(pd.Series(df[df[pid_column] == point_of_interest].dates.values[0]))
ts = np.array(df[df[pid_column] == point_of_interest].ts.values[0][band])

# plot
sns.set(rc={'figure.figsize':(25,5)})
ax = sns.scatterplot(x=dates, y=ts)
sns.lineplot(x=fake_dates, y=fake_fitted_values, ax=ax, color='orange')

# 3 Display on a map

In [ ]:
df['lat'] = df.geometry.centroid.y
df['lon'] = df.geometry.centroid.x

In [ ]:
df.head()

In [ ]:
from folium.plugins import MarkerCluster


centre = ceo_df.unary_union.convex_hull.centroid
colors = ['green', 'orange']
ceo_df['FNF_color'] = ceo_df['FNF_2017'].apply(lambda x: 'red' if x == 1 else 'blue')

# Use terrain map layer to see volcano terrain
map_ = folium.Map(location=[centre.y, centre.x], tiles="Stamen Terrain", zoom_start=8, control_scale = True)

marker_cluster = MarkerCluster(popups=ceo_df['FNF_2017']).add_to(map_)
folium.GeoJson(
    ceo_df.head(25000),
    marker=folium.Circle(
        radius=70,
        fill_color="orange",
        fill_opacity=0.75,
        color="black",
        weight=0
    )
    ,
    tooltip=folium.GeoJsonTooltip(fields=["FNF_2017", "CNC_ceo"]),
    style_function=lambda x: {'fillColor': x['properties']['FNF_color']},
    highlight_function=lambda x: {"fillOpacity": 0.95},
    zoom_on_click=True, 
    name='CEO Points'
).add_to(marker_cluster)

# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    ),
    "Bing VirtualEarth": folium.TileLayer(
        tiles="http://ecn.t3.tiles.virtualearth.net/tiles/a{q}.jpeg?g=1",
        attr="Microsoft",
        name="Bing VirtualEarth",
        overlay=True,
        control=True
    ),
}

# Add custom basemaps
#basemaps['Google Maps'].add_to(map_)
basemaps['Google Satellite'].add_to(map_)
#basemaps['Bing VirtualEarth'].add_to(map_)
basemaps['Esri Satellite'].add_to(map_)

folium.LayerControl().add_to(map_)
map_

# 4 Save file as gpkg  

In [ ]:
outfile = 'outfile.gpkg'
df.drop(['dates', 'ts'], axis=1).to_file(outfile, driver='GPKG')

# 5 Delete tmp esbae files and folders on EE manually

In [1]:
import ee
ee.Initialize()

from sampling_handler.misc import ee_helpers
ee_helpers.cleanup_tmp_esbae()


KeyboardInterrupt

